In [1]:
import serial

from clockgen import Timer, Periph, ProgramGen, hw_desc
from clockgen.interface import TIM, GPIO
from clockgen.utils import send_program, start_execution


In [14]:
ser = serial.Serial("/dev/ttyACM1", baudrate=1000000, timeout=5)

In [23]:
gp = GPIO("gpioa")
tim2 = TIM("tim3")
tim1 = TIM("d6")
tim3 = TIM("d10")

tim1.on_sync(50_000, 100)
tim2.on_sync(100_000, 100)

tim1.start_sync(120)
gp.on([1, 8], 120)
gp.off([1, 8], 150)

tim1.off_sync(1000)
tim2.off_sync(1000)
tim3.off(1500)


Found GPIOA
Found timer TIM3, pin: d4, port: pb5
Found timer TIM2, pin: d6, port: pb10
Found timer TIM4, pin: d10, port: pb6


In [24]:
pg = ProgramGen()

prog = pg.generate_program([tim1, tim2, tim3, gp])

pg.print_program()

@ 90
    TIM2->CR1           :0x40000000:0x0
    TIM2->CCMR2_Output  :0x4000001c:0x30
    TIM2->CCER          :0x40000020:0x100
    TIM2->CNT           :0x40000024:0x31f
    TIM2->ARR           :0x4000002c:0x31f
    TIM3->CR1           :0x40000400:0x0
    TIM3->CCMR1_Output  :0x40000418:0x3000
    TIM3->CCER          :0x40000420:0x10
    TIM3->CNT           :0x40000424:0x18f
    TIM3->ARR           :0x4000042c:0x18f
@ 120
    TIM1->CR1           :0x40010000:0x1
    GPIOA->ODR          :0x40020014:0x102
@ 121
    TIM1->CR1           :0x40010000:0x0
@ 150
    GPIOA->ODR          :0x40020014:0x0
@ 1000
    TIM2->CR1           :0x40000000:0x0
    TIM2->CCMR2_Output  :0x4000001c:0x40
    TIM3->CR1           :0x40000400:0x0
    TIM3->CCMR1_Output  :0x40000418:0x4000
@ 1500
    TIM4->CR1           :0x40000800:0x0
    TIM4->CCMR1_Output  :0x40000818:0x40
    TIM4->CCER          :0x40000820:0x1


In [28]:
send_program(ser, prog); start_execution(ser)